<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Pairwise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 強さ判定

出典：https://github.com/lucasmaystre/choix/blob/master/notebooks/intro-pairwise.ipynb

今日やること：不揃いの対戦結果から、強さのランキングを計算します。--- Bradely-Terry モデル 

## 準備

In [ ]:
!pip install  --trusted-host pypi.org --trusted-host files.pythonhosted.org choix networkx lxml html5lib bs4

# 一対比較による強さ判定

ここでは `choix` ライブラリを使います。一対一の対戦結果のデータがあるとします。

In [ ]:
import choix
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(precision=3, suppress=True)

`choix` では、$n$ 個の要素に $\{0, \ldots, n-1 \}$ の番号を付けます。
"$i$ が $j$ に勝った" ことを Python のタプル `(i, j)` で表します（左が勝者、右が敗者）。

簡単な例で動作確認をしてみます。

In [ ]:
n_items = 5
data = [
    (1, 0), (0, 4), (3, 1),
    (0, 2), (2, 4), (4, 3),
]

このデータをグラフとして可視化します。

- 各要素はグラフのノードに対応します。
- ノード $i$ からノード $j$ へのリンクは、"$i$ が $j$ に勝った"を示します。

In [ ]:
graph = nx.DiGraph(data)
nx.draw(graph, with_labels=True)
plt.show()

このデータを、 [Bradley-Terry model](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model) によって解析します。
`choix` はこのためのアルゴリズムを用意しています。以下では、 I-LSR と呼ばれる最尤推定法を用います。

In [ ]:
params = choix.ilsr_pairwise(n_items, data)
print(params)

パラメータ (params) は各要素の"強さ" (or utility) を表します。パラメータの値でソートすることで、強さのランキングを算出できます。

In [ ]:
print("強さランキング:", np.argsort( -params ))

## スパース性の取り扱い
グラフが連結していないと、最尤推定法が使えません。このような状況は、全勝もしは全敗の要素がある時に生じます。以下の例では、$3$ が全勝、$1$ が全敗です。

In [ ]:
n_items = 4
data = [(3, 2), (2, 1), (1, 0)]

graph = nx.DiGraph(data)
nx.draw(graph, with_labels=True)

このようなケースでは、予測に失敗します。

In [ ]:
choix.ilsr_pairwise(n_items, data)

この問題は、正規化パラメータの導入により解決できます。

In [ ]:
choix.ilsr_pairwise(n_items, data, alpha=0.01)

# 課題　大相撲で今一番強いのは？

先場所の結果から、Bradely-Terry モデルによって現時点での強さランキングを出してみましょう。強さが番付とどれくらい一致しているかを調べましょう。

In [ ]:
import pandas as pd
import time
import lxml

ヤフースポーツから結果を持ってきます。表形式のデータに限っては、pandas で取得可能。<br>
bashoId --> 年月&days=日目　で指定できます。

In [ ]:
url = "https://sports.yahoo.co.jp/sumo/torikumi/stats/?bashoId=202009&day=2"
d_list = pd.io.html.read_html(url, encoding="utf-8")

d_list[1] に幕内の結果、d_list[2] に十両の結果が入っています。

In [ ]:
d_list[1]

In [ ]:
d_list[2]

pd.concat で結合すると一つの表になります。

In [ ]:
d_lists = pd.concat([d_list[1], d_list[2]])
d_lists

１場所分まとめてとってくるには for 文を回すとできます。

In [ ]:
cols = ['東', '東.1', '東.2', '決まり手', '西', '西.1', '西.2']
df = pd.DataFrame(columns=cols)
base_url = "https://sports.yahoo.co.jp/sumo/torikumi/stats/?bashoId=202003"
for day in range(1, 16):
    print(day, "日目 ... 取得")
    url = base_url + "&days=" + str(day)
    dframe_list = pd.io.html.read_html(url, encoding="utf-8")
    tdf = pd.concat([dframe_list[1], dframe_list[2]])
    df = pd.concat([tdf, df])
    time.sleep(2)

## 課題：この場所、最強だったのは誰か？

df から対戦結果をまとめて choix 関数に適用してみてください。